Let's have a vector space $V$ of dimension $n$ and let's have a $n\times n$ matrix $A$, which acts on vectors in $V$ as $y_i = \sum_j A_{ij} x_j$, where $x_i, y_i$ are coordinates in some basis $e_i$. We want to find the matrix representation of the action of $A$ on symmetric product $S^k V$. Symmetric product is a generalization of the concept of tensor product $V^{\otimes k}$, where only symmetric combinations are included. A typical realization is the space of polynomials of multiple variables. Here $k$ is the degree of the polynomial and $n$ is the number of variables. Example: Let $V=\{x,y\}$. Then $V\otimes V$ is vector space spanned by the basis $\{(x,x), (x,y),(y,x),(y,y)\}$, while $S^2V$ is spanned by the basis $\{x^2,xy,y^2\}$. Note, that definition of tensor product requires that there is a multilinear map $b: V\times V\rightarrow V\otimes V$. For symmetric product we require that $b: V\times V\rightarrow S^2 V$ is symmetric, i.e., $b(x,y)=b(y,x)$. The symmetric product can be viewed as subspace of tensor product with basis $e_i\odot e_i = e_i \otimes e_i$ and
$e_i\odot e_j = e_i \otimes e_j+ e_j \otimes e_i$. 

In [9]:
using Combinatorics
using LinearAlgebra

# Function to generate the basis for Sym^k V
function symmetric_basis(n, k)
    # Generate all combinations of k elements from 1:n (with repetition allowed)
    return collect(with_replacement_combinations(1:n, k))
end

# Function to construct the matrix representation of A acting on Sym^k V
function matrix_action_on_symk(A, n, k)
    # Basis for Sym^k V (combinations of k elements from {1, 2, ..., n})
    basis = symmetric_basis(n, k)
    dim_sym_k = length(basis)
    
    # Initialize the matrix to store the action of A on Sym^k V
    action_matrix = zeros(Float64, dim_sym_k, dim_sym_k)

    # Convert the basis to tensor form (indexing)
    for i in 1:dim_sym_k
        for j in 1:dim_sym_k
            tensor_i = basis[i]
            right = collect(multiset_permutations(basis[j], k))
            out = 0.
            for tensor_j in right
                 # println(tensor_i,tensor_j)
                result = 1.0
                for m in 1:k
                    result *= A[tensor_i[m], tensor_j[m]]  # Action of A on the components
                end
                out +=result
                    #println(out)
            end
            action_matrix[i, j] = out
        end
    end
    return action_matrix
end


matrix_action_on_symk (generic function with 1 method)

We have $f_{i}=\sum_j A_{ij}e_j$ and we want to get 
$f_{i_1}\odot f_{i_2} \odot \dots \odot f_{i_k} = \sum_{i'_1\le i'_2\le \ldots \le i'_k} M_{i'_1,i'_2\ldots i'_k; i_1,i_2\ldots i_k}e_{i'_1}\odot e_{i'_2} \odot \dots \odot e_{i'_k}=
\sum_{i'_1\le i'_2\le \ldots \le i'_k}\sum_{\pi(i'_1,i'_2\ldots i'_k)} A_{\pi(i'_1)i_1}A_{\pi(i'_2)i_2}\ldots A_{\pi(i'_k)i_k} e_{\pi(i'_1)}\otimes e_{\pi(i'_2)} \otimes \dots \otimes e_{\pi(i'_k)}$

In [11]:
n = 2  # Dimension of V 
k = 2 # Degree of S^k V 

# Define the matrix A acting on V (n x n matrix)
A = rand(n, n)  # Random matrix representing the linear operator A on V
B = rand(n, n)  # Random matrix representing the linear operator A on V
C = A + A'    # Let's make it symmetric to have real eigenvalues



# Display the matrix A
# Get the matrix representation of A on Sym^k V
MA = matrix_action_on_symk(A, n, k);
MB = matrix_action_on_symk(B, n, k);
MAB = matrix_action_on_symk(A*B, n, k);

# Display the result
show(stdout, MIME"text/plain"(), A)
println()
show(stdout, MIME"text/plain"(), MA)

2×2 Matrix{Float64}:
 0.499848  0.566817
 0.237545  0.888096
3×3 Matrix{Float64}:
 0.249848   0.566644  0.321281
 0.118737   0.578558  0.503387
 0.0564278  0.421926  0.788714

Let's denote $f: \operatorname{End}(V)\rightarrow \operatorname{End}(S^k V)$, where $\operatorname{End}$ (for endomorphism) is the set/space of all linear maps (matrices) on $V$, resp. $S^k V$. We have just coded this $f$, i.e., we take a matrix $A$ and generate a matrix $M$. We want to check that our construction preserves the composition (matrix multiplication), i.e. that $f(AB)=f(A)f(B)$. 

In [13]:
show(stdout, MIME"text/plain"(), MAB)
println()
show(stdout, MIME"text/plain"(), MA*MB-MAB)

3×3 Matrix{Float64}:
 0.389133  0.714241  0.327741
 0.468342  0.835617  0.37242
 0.563674  0.976812  0.423189
3×3 Matrix{Float64}:
 -1.11022e-16  -1.11022e-16  -1.11022e-16
 -1.11022e-16   0.0          -5.55112e-17
 -1.11022e-16   0.0          -5.55112e-17

In [14]:
a = eigvals(C)  # eigenvalues of C

2-element Vector{Float64}:
 0.49478417920201906
 2.2811038683945193

In [15]:
eigvals(matrix_action_on_symk(C, n, k)) # eigenvalues of f(C)

3-element Vector{Float64}:
 0.24481138398861554
 1.128654105198133
 5.203434858404436

How to calculate eigenvalues of $M$? We use the fact that $f(A'=TAT^{-1})=f(T)f(A)f(T)^{-1}$ and diagonalize $A$. In the new basis
we get $f(A')f_{i_1}\odot f_{i_2} \odot \dots \odot f_{i_k}=A'f_{i_1}\odot A'f_{i_2} \odot \dots \odot A'f_{i_k}=
\lambda_{i_1}\lambda_{i_2}\ldots \lambda_{i_k}f_{i_1}\odot f_{i_2} \odot \dots \odot f_{i_k}$.

In [17]:
function l_power(e::Vector, p::Vector{Int})
    out = 1.0
    for i in p
        out *=e[i]
    end
    out
end

lps(e::Vector, basis::Vector{Vector{Int}}) = map(x-> l_power(e, x), basis) 
    

lps (generic function with 1 method)

In [18]:
basis = symmetric_basis(n,k)

3-element Vector{Vector{Int64}}:
 [1, 1]
 [1, 2]
 [2, 2]

In [19]:
lps(a,basis)

3-element Vector{Float64}:
 0.2448113839886157
 1.1286541051981327
 5.2034348584044405

How to calculate trace, $\operatorname{Tr}S^k A$? For tensor product the trace of $\operatorname{Tr} A^{\otimes k}=(\operatorname{Tr} A)^k=\sum_{i_1,i_2,\ldots i_k}\lambda_{i_1}\lambda_{i_2}\ldots \lambda_{i_k} =(\sum_{i}\lambda_{i})^k$. For $S^k A$ the sums are not free and we have $\operatorname{Tr}S^k A = \sum_{i_1\le i_2\le \ldots \le i_k}\lambda_{i_1}\lambda_{i_2}\ldots \lambda_{i_k}$. For $k=2$ we can find a special formula $\sum_{i\le j}\lambda_{i}\lambda_{j}=\tfrac{1}{2}(\sum_{i\,j}\lambda_{i}\lambda_{j}+\sum_{i}\lambda^2_{i})=\tfrac{1}{2}((\operatorname{Tr}A)^2+\operatorname{Tr}(A^2))$.

In [21]:
n = 4  # Dimension of V 
k = 2 # Degree of S^k V 

# Define the matrix A acting on V (n x n matrix)
A = rand(n, n)  # Random matrix representing the linear operator A on V

C = A + A'    # Let's make it symmetric to have real eigenvalues
MC = matrix_action_on_symk(C, n, k);

println(tr(C))
println((tr(C)^2+tr(C^2))/2)
tr(MC)


4.717590846258696
21.030538603988816


21.030538603988816

In [22]:
n = 2
k = 3
A = [-1/2 -sqrt(3)/2; sqrt(3)/2 -1/2]
B = [ 1 0; 0 -1]
matrix_action_on_symk(A, n, k)

4×4 Matrix{Float64}:
 -0.125     -0.649519  -1.125     -0.649519
  0.216506   0.625      0.216506  -0.375
 -0.375     -0.216506   0.625     -0.216506
  0.649519  -1.125      0.649519  -0.125

Matrix representation of $D_3$ in the $V=(x,y)$ basis ($z\parallel C_3$).

In [24]:
A = [-1/2 -sqrt(3)/2; sqrt(3)/2 -1/2]
B = [ 1 0; 0 -1]
g = Vector{Matrix}()
push!(g, B*B)
push!(g, A)
push!(g, A*A)
push!(g,B)
push!(g,B*A)
push!(g,B*A*A)


6-element Vector{Matrix}:
 [1 0; 0 1]
 [-0.5 -0.8660254037844386; 0.8660254037844386 -0.5]
 [-0.4999999999999999 0.8660254037844386; -0.8660254037844386 -0.4999999999999999]
 [1 0; 0 -1]
 [-0.5 -0.8660254037844386; -0.8660254037844386 0.5]
 [-0.4999999999999999 0.8660254037844386; 0.8660254037844386 0.4999999999999999]

$\operatorname{Sym}^k V$ representation of $D_3$. Note that if we view the basis functions as polynomials they have the 
form $\pmatrix{k \\ l}x^{l}y^{k-l}$, i.e., a binomial prefactor.

In [26]:
sym_rep(x,k)=map(y->matrix_action_on_symk(y, size(y)[1], k),x)

sym_rep (generic function with 1 method)

Character table of $D_3$.

In [28]:
Table = [[1,1,1,1,1,1],[1,1,1,-1,-1,-1],[2,-1,-1,0,0,0]]

3-element Vector{Vector{Int64}}:
 [1, 1, 1, 1, 1, 1]
 [1, 1, 1, -1, -1, -1]
 [2, -1, -1, 0, 0, 0]

Projection operators on the irreps.

In [30]:
Projections = Vector{Matrix}()
group = sym_rep(g,3)
for irrep in Table
    P=sum(irrep.*group)*irrep[1]/length(group)
    show(IOContext(stdout), MIME"text/plain"(), P)
    push!(Projections,P)
    println()
end

4×4 Matrix{Float64}:
  0.25  -1.4803e-16   -0.75   0.0
  0.0    5.55112e-17   0.0    9.25186e-18
 -0.25   7.40149e-17   0.75  -3.70074e-17
  0.0    3.70074e-17   0.0    4.62593e-18
4×4 Matrix{Float64}:
 4.62593e-18   0.0    3.70074e-17   0.0
 2.77556e-17   0.75  -7.40149e-17  -0.25
 0.0           0.0    5.55112e-17   0.0
 0.0          -0.75   1.11022e-16   0.25
4×4 Matrix{Float64}:
  0.75          1.4803e-16    0.75         0.0
 -2.77556e-17   0.25          7.40149e-17  0.25
  0.25         -7.40149e-17   0.25         3.70074e-17
  0.0           0.75         -1.11022e-16  0.75


Basis of $A_1$ prepresentation on $\operatorname{Sym}^3 V$, i.e. $x^3-3 xy^2$. The basis order is $\{x^3,x^2y,xy^2,y^3\}$. Note that
there are $xy^2$ corresponds to 3 elements of the symmetric tensor $xyy$, $yxy$ and $yyx$. Therefore the factor of 3 in the polynomial. 

In [32]:
eigvecs(Projections[1])[:,4]

4-element Vector{Float64}:
  0.7071067811865475
  0.0
 -0.7071067811865476
  0.0

In [33]:
# labeling of the basis states of Sym^3 V.
symmetric_basis(2,3)

4-element Vector{Vector{Int64}}:
 [1, 1, 1]
 [1, 1, 2]
 [1, 2, 2]
 [2, 2, 2]

$(x,y)$ representation of $D_{3d}=D_3\otimes C_i$: Here A is one of the $C_3$ rotations and B is the $C_2$ rotation about x. The other elements are obtained by mulitplation. Note that the order of the elements matters, must match the character table (taken from https://symmetry.constructor.university)! 

In [35]:
A = [-1/2 -sqrt(3)/2; sqrt(3)/2 -1/2]
B = [ 1 0; 0 -1]
g2 = Vector{Matrix}()
push!(g2, B*B)
push!(g2, A)
push!(g2, A*A)
push!(g2,B)
push!(g2,B*A)
push!(g2,B*A*A)
push!(g2, -B*B)
push!(g2, -A)
push!(g2, -A*A)
push!(g2,-B)
push!(g2,-B*A)
push!(g2,-B*A*A)

12-element Vector{Matrix}:
 [1 0; 0 1]
 [-0.5 -0.8660254037844386; 0.8660254037844386 -0.5]
 [-0.4999999999999999 0.8660254037844386; -0.8660254037844386 -0.4999999999999999]
 [1 0; 0 -1]
 [-0.5 -0.8660254037844386; -0.8660254037844386 0.5]
 [-0.4999999999999999 0.8660254037844386; 0.8660254037844386 0.4999999999999999]
 [-1 0; 0 -1]
 [0.5 0.8660254037844386; -0.8660254037844386 0.5]
 [0.4999999999999999 -0.8660254037844386; 0.8660254037844386 0.4999999999999999]
 [-1 0; 0 1]
 [0.5 0.8660254037844386; 0.8660254037844386 -0.5]
 [0.4999999999999999 -0.8660254037844386; -0.8660254037844386 -0.4999999999999999]

$(xz,yz)$ representation of $D_{3d}$:

In [37]:
A = [-1/2 -sqrt(3)/2; sqrt(3)/2 -1/2]
B = [ 1 0; 0 -1]
g3 = Vector{Matrix}()
push!(g3, B*B)
push!(g3, A)
push!(g3, A*A)
push!(g3,-B)
push!(g3,-B*A)
push!(g3,-B*A*A)
push!(g3, B*B)
push!(g3, A)
push!(g3, A*A)
push!(g3,-B)
push!(g3,-B*A)
push!(g3,-B*A*A)

12-element Vector{Matrix}:
 [1 0; 0 1]
 [-0.5 -0.8660254037844386; 0.8660254037844386 -0.5]
 [-0.4999999999999999 0.8660254037844386; -0.8660254037844386 -0.4999999999999999]
 [-1 0; 0 1]
 [0.5 0.8660254037844386; 0.8660254037844386 -0.5]
 [0.4999999999999999 -0.8660254037844386; -0.8660254037844386 -0.4999999999999999]
 [1 0; 0 1]
 [-0.5 -0.8660254037844386; 0.8660254037844386 -0.5]
 [-0.4999999999999999 0.8660254037844386; -0.8660254037844386 -0.4999999999999999]
 [-1 0; 0 1]
 [0.5 0.8660254037844386; 0.8660254037844386 -0.5]
 [0.4999999999999999 -0.8660254037844386; -0.8660254037844386 -0.4999999999999999]

$(R_x,R_y)$ representation of $D_{3d}$:

In [39]:
A = [-1/2 -sqrt(3)/2; sqrt(3)/2 -1/2]
B = [ 1 0; 0 -1]
g2R = Vector{Matrix}()
push!(g2R, B*B)
push!(g2R, A)
push!(g2R, A*A)
push!(g2R,B)
push!(g2R,B*A)
push!(g2R,B*A*A)
push!(g2R, B*B)
push!(g2R, A)
push!(g2R, A*A)
push!(g2R,B)
push!(g2R,B*A)
push!(g2R,B*A*A)

12-element Vector{Matrix}:
 [1 0; 0 1]
 [-0.5 -0.8660254037844386; 0.8660254037844386 -0.5]
 [-0.4999999999999999 0.8660254037844386; -0.8660254037844386 -0.4999999999999999]
 [1 0; 0 -1]
 [-0.5 -0.8660254037844386; -0.8660254037844386 0.5]
 [-0.4999999999999999 0.8660254037844386; 0.8660254037844386 0.4999999999999999]
 [1 0; 0 1]
 [-0.5 -0.8660254037844386; 0.8660254037844386 -0.5]
 [-0.4999999999999999 0.8660254037844386; -0.8660254037844386 -0.4999999999999999]
 [1 0; 0 -1]
 [-0.5 -0.8660254037844386; -0.8660254037844386 0.5]
 [-0.4999999999999999 0.8660254037844386; 0.8660254037844386 0.4999999999999999]

Character table of $D_{3d}$: This is an expanded table, i.e., characters are listed for each element, not only each class.

In [41]:
Table2 = [[1,1,1,1,1,1,1,1,1,1,1,1],
    [1,1,1,-1,-1,-1,1,1,1,-1,-1,-1],
    [2,-1,-1,0,0,0,2,-1,-1,0,0,0],
     [1,1,1,1,1,1,-1,-1,-1,-1,-1,-1],
     [1,1,1,-1,-1,-1,-1,-1,-1,1,1,1],
     [2,-1,-1,0,0,0,-2,1,1,0,0,0]]

6-element Vector{Vector{Int64}}:
 [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
 [1, 1, 1, -1, -1, -1, 1, 1, 1, -1, -1, -1]
 [2, -1, -1, 0, 0, 0, 2, -1, -1, 0, 0, 0]
 [1, 1, 1, 1, 1, 1, -1, -1, -1, -1, -1, -1]
 [1, 1, 1, -1, -1, -1, -1, -1, -1, 1, 1, 1]
 [2, -1, -1, 0, 0, 0, -2, 1, 1, 0, 0, 0]

Representation of $D_{3d}$ on the basis of symmetric product $\operatorname{Sym}^3 (x,y)$ - This is just a test, can be compared with cubic polynomials representations in the tables. Projections[l] is the projection operator from this representation to the l's irrep.

In [43]:
Projections = Vector{Matrix}()
group = sym_rep(g2,3)
for irrep in Table2
    P=sum(irrep.*group)*irrep[1]/length(group)
    show(IOContext(stdout), MIME"text/plain"(), P)
    push!(Projections,P)
    println()
end

4×4 Matrix{Float64}:
 -1.61908e-17  0.0          5.55112e-17   0.0
  0.0          4.62593e-17  0.0           4.62593e-18
  0.0          0.0          2.77556e-17   0.0
  0.0          1.85037e-17  0.0          -2.31296e-18
4×4 Matrix{Float64}:
 -2.31296e-18  0.0          1.85037e-17   0.0
  0.0          2.77556e-17  0.0           1.38778e-17
  0.0          0.0          4.62593e-17   0.0
  0.0          5.55112e-17  0.0          -1.61908e-17
4×4 Matrix{Float64}:
 1.38778e-17   0.0          -3.70074e-17   0.0
 0.0          -1.85037e-17   0.0          -9.25186e-18
 0.0           0.0          -1.85037e-17   0.0
 0.0          -3.70074e-17   0.0           1.38778e-17
4×4 Matrix{Float64}:
  0.25  -1.4803e-16   -0.75   0.0
  0.0    2.77556e-17   0.0    4.62593e-18
 -0.25   7.40149e-17   0.75  -3.70074e-17
  0.0    1.85037e-17   0.0    2.31296e-18
4×4 Matrix{Float64}:
 2.31296e-18   0.0    1.85037e-17   0.0
 2.77556e-17   0.75  -7.40149e-17  -0.25
 0.0           0.0    2.77556e-17   0.0
 0.0      

In [44]:
eigvecs(Projections[4])[:,4]  # the same as for D3 (consistency check)

4-element Vector{Float64}:
  0.7071067811865475
  0.0
 -0.7071067811865476
  0.0

Representation of $D_{3d}$ on the basis of the tensor product $(R_x,R_y)\otimes(xz,yz)$ - This is actual calculation. Note that this is 
4D rerpesentation. Projections[l] is the projection operator from this representation to the l's irrep.

In [46]:
Projections = Vector{Matrix}()
group = kron.(g2R,g3)  #sym_rep(g2,3)
for irrep in Table2
    P=sum(irrep.*group)*irrep[1]/length(group)
    show(IOContext(stdout), MIME"text/plain"(), P)
    push!(Projections,P)
    println()
end

4×4 Matrix{Float64}:
  9.25186e-18   0.0   0.0  0.0
 -3.70074e-17   0.5  -0.5  3.70074e-17
 -3.70074e-17  -0.5   0.5  3.70074e-17
  0.0           0.0   0.0  9.25186e-18
4×4 Matrix{Float64}:
 0.5   3.70074e-17   3.70074e-17  0.5
 0.0   9.25186e-18   0.0          0.0
 0.0   0.0           9.25186e-18  0.0
 0.5  -3.70074e-17  -3.70074e-17  0.5
4×4 Matrix{Float64}:
  0.5          -3.70074e-17  -3.70074e-17  -0.5
  3.70074e-17   0.5           0.5          -3.70074e-17
  3.70074e-17   0.5           0.5          -3.70074e-17
 -0.5           3.70074e-17   3.70074e-17   0.5
4×4 Matrix{Float64}:
 -9.25186e-18  0.0          0.0           0.0
  0.0          9.25186e-18  1.85037e-17   0.0
  0.0          1.85037e-17  9.25186e-18   0.0
  0.0          0.0          0.0          -9.25186e-18
4×4 Matrix{Float64}:
  9.25186e-18   0.0           0.0          -1.85037e-17
  0.0          -9.25186e-18   0.0           0.0
  0.0           0.0          -9.25186e-18   0.0
 -1.85037e-17   0.0           0.0          

Basis order $\{R_x\otimes xz, R_x\otimes yz, R_y\otimes xz, R_y\otimes xz\}$.

In [48]:
l=1   # l=1 corresponds to A_1g
println(eigvals(Projections[l]))
eigvecs(Projections[l])[:,4]   # 4th vector corresponds to the eigenvalue 1 and is the 1D basis of $A_{1g}$.

[0.0, 9.25185853854297e-18, 9.25185853854297e-18, 0.9999999999999999]


4-element Vector{Float64}:
  0.0
 -0.7071067811865475
  0.7071067811865475
  0.0

Representation of $D_{3d}$ on the basis of $(R_x,R_y)\otimes\operatorname{Sym}^2 (x,y)$ - This is actual calculation. Note that this is a 6D representation. Projections[l] is the projection operator from this representation to the l's irrep.

In [50]:
Projections = Vector{Matrix}()
group = kron.(g2R,sym_rep(g2,2))  #sym_rep(g2,3)
for irrep in Table2
    P=sum(irrep.*group)*irrep[1]/length(group)
    show(IOContext(stdout), MIME"text/plain"(), P)
    push!(Projections,P)
    println()
end

6×6 Matrix{Float64}:
  0.25  -7.40149e-17  -0.25  -2.77556e-17  -0.5   0.0
  0.0    9.25186e-18   0.0    0.0           0.0   0.0
 -0.25   7.40149e-17   0.25   0.0           0.5  -2.77556e-17
  0.0    0.0           0.0    2.31296e-18   0.0   4.62593e-18
 -0.25   3.70074e-17   0.25   3.70074e-17   0.5  -3.70074e-17
  0.0    0.0           0.0    4.62593e-18   0.0   2.31296e-18
6×6 Matrix{Float64}:
 2.31296e-18   0.0   4.62593e-18   0.0    0.0           0.0
 3.70074e-17   0.5  -3.70074e-17   0.25  -3.70074e-17  -0.25
 4.62593e-18   0.0   2.31296e-18   0.0    0.0           0.0
 2.77556e-17   0.5   0.0           0.25  -7.40149e-17  -0.25
 0.0           0.0   0.0           0.0    9.25186e-18   0.0
 0.0          -0.5   2.77556e-17  -0.25   7.40149e-17   0.25
6×6 Matrix{Float64}:
  0.75          7.40149e-17   0.25          2.77556e-17   0.5          0.0
 -3.70074e-17   0.5           3.70074e-17  -0.25          3.70074e-17  0.25
  0.25         -7.40149e-17   0.75          0.0          -0.5      

Basis order $\{R_x\otimes x^2, R_x\otimes xy, R_x\otimes y^2, R_y\otimes x^2, R_y\otimes xy, R_y\otimes y^2\}$.

In [52]:
l=1    # l=1  l=1 corresponds to A_1g
println(eigvals(Projections[l]))
eigvecs(Projections[l])[:,6]   # 6th vector corresponds to the eigenvalue 1

ComplexF64[0.0 + 0.0im, 2.1985277227013376e-18 + 0.0im, 9.25185853854297e-18 + 0.0im, 3.001301601718574e-17 - 6.228609377035466e-17im, 3.001301601718574e-17 + 6.228609377035466e-17im, 0.9999999999999998 + 0.0im]


6-element Vector{ComplexF64}:
     -0.5773502691896258 + 0.0im
                     0.0 + 0.0im
      0.5773502691896258 + 0.0im
   -3.84418110305865e-19 + 0.0im
      0.5773502691896258 + 0.0im
 -5.0824956067723185e-18 + 0.0im

$(x,y)$ representation of $C_{3v}$: Here A is one of the $C_3$ rotations and B is the mirror perp to y. The other elements are obtained by mulitplation. Note that the order of the elements matters, must match the character table!

In [54]:
A = [-1/2 -sqrt(3)/2; sqrt(3)/2 -1/2]
B = [ 1 0; 0 -1]
g2 = Vector{Matrix}()
push!(g2, B*B)
push!(g2, A)
push!(g2, A*A)
push!(g2,B)
push!(g2,B*A)
push!(g2,B*A*A)

6-element Vector{Matrix}:
 [1 0; 0 1]
 [-0.5 -0.8660254037844386; 0.8660254037844386 -0.5]
 [-0.4999999999999999 0.8660254037844386; -0.8660254037844386 -0.4999999999999999]
 [1 0; 0 -1]
 [-0.5 -0.8660254037844386; -0.8660254037844386 0.5]
 [-0.4999999999999999 0.8660254037844386; 0.8660254037844386 0.4999999999999999]

$(xz,yz)$ representation of $C_{3v}$:

In [56]:
A = [-1/2 -sqrt(3)/2; sqrt(3)/2 -1/2]
B = [ 1 0; 0 -1]
g3 = Vector{Matrix}()
push!(g3, B*B)
push!(g3, A)
push!(g3, A*A)
push!(g3,B)
push!(g3,B*A)
push!(g3,B*A*A)

6-element Vector{Matrix}:
 [1 0; 0 1]
 [-0.5 -0.8660254037844386; 0.8660254037844386 -0.5]
 [-0.4999999999999999 0.8660254037844386; -0.8660254037844386 -0.4999999999999999]
 [1 0; 0 -1]
 [-0.5 -0.8660254037844386; -0.8660254037844386 0.5]
 [-0.4999999999999999 0.8660254037844386; 0.8660254037844386 0.4999999999999999]

$(R_x,R_y)$ representation of $C_{3v}$:

In [58]:
A = [-1/2 -sqrt(3)/2; sqrt(3)/2 -1/2]
B = [ 1 0; 0 -1]
g2R = Vector{Matrix}()
push!(g2R, B*B)
push!(g2R, A)
push!(g2R, A*A)
push!(g2R,-B)
push!(g2R,-B*A)
push!(g2R,-B*A*A)

6-element Vector{Matrix}:
 [1 0; 0 1]
 [-0.5 -0.8660254037844386; 0.8660254037844386 -0.5]
 [-0.4999999999999999 0.8660254037844386; -0.8660254037844386 -0.4999999999999999]
 [-1 0; 0 1]
 [0.5 0.8660254037844386; 0.8660254037844386 -0.5]
 [0.4999999999999999 -0.8660254037844386; -0.8660254037844386 -0.4999999999999999]

Representation of $C_{3v}$ on the basis of $(R_x,R_y)\otimes\operatorname{Sym}^2 (x,y)$ - This is actual calculation. Note that this is a 6D representation.

In [60]:
Table2= [[1,1,1,1,1,1],[1,1,1,-1,-1,-1],[2,-1,-1,0,0,0]]
Projections = Vector{Matrix}()
group = kron.(g2R,sym_rep(g2,2))  #sym_rep(g2,3)
for irrep in Table2
    P=sum(irrep.*group)*irrep[1]/length(group)
    show(IOContext(stdout), MIME"text/plain"(), P)
    push!(Projections,P)
    println()
end

6×6 Matrix{Float64}:
 4.62593e-18   0.0   9.25186e-18   0.0    0.0           0.0
 3.70074e-17   0.5  -3.70074e-17   0.25  -3.70074e-17  -0.25
 9.25186e-18   0.0   4.62593e-18   0.0    0.0           0.0
 2.77556e-17   0.5   0.0           0.25  -7.40149e-17  -0.25
 0.0           0.0   0.0           0.0    1.85037e-17   0.0
 0.0          -0.5   2.77556e-17  -0.25   7.40149e-17   0.25
6×6 Matrix{Float64}:
  0.25  -7.40149e-17  -0.25  -2.77556e-17  -0.5   0.0
  0.0    1.85037e-17   0.0    0.0           0.0   0.0
 -0.25   7.40149e-17   0.25   0.0           0.5  -2.77556e-17
  0.0    0.0           0.0    4.62593e-18   0.0   9.25186e-18
 -0.25   3.70074e-17   0.25   3.70074e-17   0.5  -3.70074e-17
  0.0    0.0           0.0    9.25186e-18   0.0   4.62593e-18
6×6 Matrix{Float64}:
  0.75          7.40149e-17   0.25          2.77556e-17   0.5          0.0
 -3.70074e-17   0.5           3.70074e-17  -0.25          3.70074e-17  0.25
  0.25         -7.40149e-17   0.75          0.0          -0.5      

In [61]:
l=1    # l=1  l=1 corresponds to A_1g
println(eigvals(Projections[l]))
eigvecs(Projections[l])[:,6]   # 6th vector corresponds to the eigenvalue 1

ComplexF64[4.625929269271485e-18 + 0.0im, 1.850371707708594e-17 + 0.0im, 6.132067109110266e-17 - 1.8214078991720256e-17im, 6.132067109110266e-17 + 1.8214078991720256e-17im, 2.220446049250313e-16 + 0.0im, 0.9999999999999999 + 0.0im]


6-element Vector{ComplexF64}:
 1.4294351254247373e-34 + 0.0im
      0.577350269189626 + 0.0im
 1.7166942661901716e-17 + 0.0im
     0.5773502691896257 + 0.0im
                    0.0 + 0.0im
    -0.5773502691896256 + 0.0im

Representation of $C_{3v}$ on the basis of the tensor product $(R_x,R_y)\otimes(xz,yz)$ - This is actual calculation. Note that this is 
4D rerpesentation.

In [63]:
Table2= [[1,1,1,1,1,1],[1,1,1,-1,-1,-1],[2,-1,-1,0,0,0]]
Projections = Vector{Matrix}()
group = kron.(g2R,g2)  #sym_rep(g2,3)
for irrep in Table2
    P=sum(irrep.*group)*irrep[1]/length(group)
    show(IOContext(stdout), MIME"text/plain"(), P)
    push!(Projections,P)
    println()
end

4×4 Matrix{Float64}:
  1.85037e-17   0.0   0.0  0.0
 -3.70074e-17   0.5  -0.5  3.70074e-17
 -3.70074e-17  -0.5   0.5  3.70074e-17
  0.0           0.0   0.0  1.85037e-17
4×4 Matrix{Float64}:
 0.5   3.70074e-17   3.70074e-17  0.5
 0.0   1.85037e-17   0.0          0.0
 0.0   0.0           1.85037e-17  0.0
 0.5  -3.70074e-17  -3.70074e-17  0.5
4×4 Matrix{Float64}:
  0.5          -3.70074e-17  -3.70074e-17  -0.5
  3.70074e-17   0.5           0.5          -3.70074e-17
  3.70074e-17   0.5           0.5          -3.70074e-17
 -0.5           3.70074e-17   3.70074e-17   0.5


In [64]:
l=1    # l=1  l=1 corresponds to A_1g
println(eigvals(Projections[l]))
eigvecs(Projections[l])[:,4]   # 6th vector corresponds to the eigenvalue 1

[0.0, 1.850371707708594e-17, 1.850371707708594e-17, 0.9999999999999999]


4-element Vector{Float64}:
  0.0
 -0.7071067811865475
  0.7071067811865475
  0.0

In [65]:
Table2= [[1,1,1,1,1,1],[1,1,1,-1,-1,-1],[2,-1,-1,0,0,0]]
Projections = Vector{Matrix}()
group = kron.(g2R,g2R)  #sym_rep(g2,3)
for irrep in Table2
    P=sum(irrep.*group)*irrep[1]/length(group)
    show(IOContext(stdout), MIME"text/plain"(), P)
    push!(Projections,P)
    println()
end

4×4 Matrix{Float64}:
 0.5   3.70074e-17   3.70074e-17  0.5
 0.0   1.85037e-17   0.0          0.0
 0.0   0.0           1.85037e-17  0.0
 0.5  -3.70074e-17  -3.70074e-17  0.5
4×4 Matrix{Float64}:
  1.85037e-17   0.0   0.0  0.0
 -3.70074e-17   0.5  -0.5  3.70074e-17
 -3.70074e-17  -0.5   0.5  3.70074e-17
  0.0           0.0   0.0  1.85037e-17
4×4 Matrix{Float64}:
  0.5          -3.70074e-17  -3.70074e-17  -0.5
  3.70074e-17   0.5           0.5          -3.70074e-17
  3.70074e-17   0.5           0.5          -3.70074e-17
 -0.5           3.70074e-17   3.70074e-17   0.5


In [66]:
l=2    # l=1  l=1 corresponds to A_1g
println(eigvals(Projections[l]))
eigvecs(Projections[l])[:,4]   # 6th vector corresponds to the eigenvalue 1

[0.0, 1.850371707708594e-17, 1.850371707708594e-17, 0.9999999999999999]


4-element Vector{Float64}:
  0.0
 -0.7071067811865475
  0.7071067811865475
  0.0